First, let's import all the packages we will need for this project

In [2]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

Configure pandas

In [24]:
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_rows', 250)
pd.set_option('display.min_rows', None)

Filters

In [ ]:
yesterday = datetime.date(datetime.today() - timedelta(1))
filter_date = data['Date'] == str(yesterday)
filter_country = data['Country'] == 'United States of America'
filter_province = data['Province'] == ''

To see the disease development over time, let's access the Covid-19 data API kindly provided by the [Johns Hopkins CSSE](https://covid19api.com/).

[Postman COVID-19 API Resource Center](https://covid-19-apis.postman.com/)

In [ ]:
def getInstructions():
    
    r = requests.get('https://api.covid19api.com/')
    instructions_dic = r.json()
    instructions = pd.DataFrame.from_dict(instructions_dic).transpose()
    
    return instructions

In [55]:
def timeSeriesData(list_countries = ['estonia', 'latvia', 'lithuania']):

    data = []

    for country in list_countries:
        r = requests.get(f'https://api.covid19api.com/country/{country}')
        data_dic = r.json()
        raw_data = pd.DataFrame.from_dict(data_dic)

        filter_province = raw_data['Province'] != ''
        raw_data.drop(index = raw_data[filter_province].index, inplace = True)

        filter_columns = ['CountryCode', 'Province', 'City', 'CityCode', 'Lat', 'Lon', 'Active']
        raw_data.drop(columns = filter_columns, inplace = True)

        raw_data['Date'] = pd.to_datetime(raw_data['Date'], format = '%Y-%m-%dT%H:%M:%SZ')
        raw_data[['Confirmed', 'Deaths', 'Recovered']] = raw_data[['Confirmed', 'Deaths', 'Recovered']].apply(pd.to_numeric)

        data.append(raw_data)

    data = pd.concat(data)

    column_names = ['Country', 'Date', 'Confirmed', 'Deaths', 'Recovered']
    data = data.reindex(columns = column_names)

    filter_date = data['Date'] >= '2020-03-01'
    data = data[filter_date]

    country_group = data.groupby('Country')

    return country_group

In [67]:
data = timeSeriesData()
data.get_group('Lithuania')


,Country,Date,Confirmed,Deaths,Recovered
39,Lithuania,2020-03-01,1,0,0
40,Lithuania,2020-03-02,1,0,0
41,Lithuania,2020-03-03,1,0,0
42,Lithuania,2020-03-04,1,0,0
43,Lithuania,2020-03-05,1,0,0
44,Lithuania,2020-03-06,1,0,0
45,Lithuania,2020-03-07,1,0,0
46,Lithuania,2020-03-08,1,0,0
47,Lithuania,2020-03-09,1,0,0
48,Lithuania,2020-03-10,1,0,0


To see more insights of the current situation, let's extract cross-sectional data from the [NovelCOVID API](https://github.com/javieraviles/covidAPI)

In [22]:
r = requests.get('https://coronavirus-19-api.herokuapp.com/countries')
cs_data_dic = r.json()
cs_data = pd.DataFrame.from_dict(cs_data_dic)

In [27]:
cs_data = cs_data.loc[8:218]
filter_columns = ['CountryCode', 'Province', 'City', 'CityCode', 'Lat', 'Lon', 'Active']
cs_data.drop(columns = ['CountryCode', 'Province', 'City', 'CityCode', 'Lat', 'Lon', 'Active'], inplace = True)

,country,cases,todayCases,deaths,todayDeaths,recovered,active,critical,casesPerOneMillion,deathsPerOneMillion,totalTests,testsPerOneMillion
8,USA,925758,526,52217,24,110432.0,763109,15097,2797,158,5037473,15219
9,Spain,219764,0,22524,0,92355.0,104885,7705,4700,482,930230,19896
10,Italy,192994,0,25969,0,60498.0,106527,2173,3192,430,1642356,27164
11,France,159828,0,22245,0,43493.0,94090,4870,2449,341,463662,7103
12,Germany,154999,0,5760,0,109800.0,39439,2908,1850,69,2072669,24738
13,UK,143464,0,19506,0,NaN,123614,1559,2113,287,612031,9016
14,Turkey,104912,0,2600,0,21737.0,80575,1790,1244,31,830257,9844
15,Iran,88194,0,5574,0,66599.0,16021,3121,1050,66,399927,4761
16,Russia,68622,0,615,0,5568.0,62439,2300,470,4,2550000,17474
17,Brazil,54043,1048,3704,34,27655.0,22684,8318,254,17,291922,1373
